In [63]:
%config IPCompleter.greedy=True
import lmdb
import os
import csv
import pickle
from matplotlib import pyplot as plt
import numpy as np #il faut enlever np
#import matplotlib.pyplot as plt, mpld3
from matplotlib import style
import math

In [3]:
path_file = "2014-01 - Citi Bike trip data.csv"
chunk_size = 100

In [9]:
lmdb_database = lmdb.open('bike_db', map_size = 2 ** 30)
#x = pickle.dumps(1)
#print(x)
#print(pickle.loads(x))

In [5]:
row = [
        "tripduration","starttime","stoptime","start station id","start station name",
        "start station latitude","start station longitude",
        "end station id","end station name",
        "end station latitude","end station longitude"
        ]

In [50]:
with open(path_file) as csv_file:
    reader = csv.reader(csv_file)
    next(reader)
    line_number = 1
    with lmdb_database.begin(write = True) as txn:
        for row in reader :
            notBinaryObject = row[0:11]
            txn.put(pickle.dumps(line_number), pickle.dumps(notBinaryObject))
            line_number = line_number + 1
            if(line_number == 100000) : 
                break
            #break
        print(line_number)

100000


In [68]:
with lmdb_database.begin() as txn:
    cursor = txn.cursor()
    plt.rcParams['figure.figsize'] = (16, 9)
    plt.style.use('ggplot')
    data = [] #initialisation d'un tableau pour tester un kmeans avec les start long et lat 
    for key, value in cursor:
        data.append([pickle.loads(value)[6],pickle.loads(value)[7]])
    print(data[0])

['-73.99510132', '438']


In [55]:
#with open(path_file) as csv_file:
#    reader = csv.reader(csv_file)
#    next(reader)
#    data = [r for r in reader]
#    print(data[0][0:11])
#    print(pickle.dumps(data[0][0:11]))

In [70]:
#trouvé sur https://github.com/madhug-nadig/Machine-Learning-Algorithms-from-Scratch/blob/master/K%20Means%20Clustering.py
#pas testé 

#================================================================================================================
#
#                                                K MEANS CLUSTERING
#
#================================================================================================================

style.use('ggplot')

class K_Means:
    def __init__(self, k =3, tolerance = 0.0001, max_iterations = 500):
        self.k = k
        self.tolerance = tolerance
        self.max_iterations = max_iterations

    def Euclidean_distance(feat_one, feat_two):

        squared_distance = 0
        #Assuming correct input to the function where the lengths of two features are the same

        for i in range(len(feat_one)):
                squared_distance += (feat_one[i] - feat_two[i])**2

        ed = sqrt(squared_distances)
        return ed;

    def fit(self, data):

        self.centroids = {}

        #initialize the centroids, the first 'k' elements in the dataset will be our initial centroids
        for i in range(self.k):
            self.centroids[i] = data[i]

        #begin iterations
        for i in range(self.max_iterations):
            self.classes = {}
            for i in range(self.k):
                self.classes[i] = []

        #find the distance between the point and cluster; choose the nearest centroid
            for features in data:
                distances = [Euclidean_distance(features,self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classes[classification].append(features)

            previous = dict(self.centroids)

            #average the cluster datapoints to re-calculate the centroids
            for classification in self.classes:
                self.centroids[classification] = np.average(self.classes[classification], axis = 0)
                #il faut remplacer np

            isOptimal = True

            for centroid in self.centroids:

                original_centroid = previous[centroid]
                curr = self.centroids[centroid]
                
                #il faut remplacer np
                if np.sum((curr - original_centroid)/original_centroid * 100.0) > self.tolerance:
                    isOptimal = False

            #break out of the main loop if the results are optimal, ie. the centroids don't change their positions much(more than our tolerance)
            if isOptimal:
                break

    def pred(self, data):
        distances = [Euclidean_distance(data,self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

    

In [69]:
km = K_Means(3)
km.fit(data)

# Plotting starts here
colors = 10*["r", "g", "c", "b", "k"]

for centroid in km.centroids:
    plt.scatter(km.centroids[centroid][0], km.centroids[centroid][1], s = 130, marker = "x")

    for classification in km.classes:
        color = colors[classification]
        for features in km.classes[classification]:
            plt.scatter(features[0], features[1], color = color,s = 30)
            
    mpld3.show()


NameError: name 'Euclidean_distance' is not defined

In [67]:
data[0]

['-73.99510132',
 '-73.98014437',
 '-73.98788205',
 '-73.98338988',
 '-74.00443172',
 '-73.97768752',
 '-73.98165557',
 '-73.97648516',
 '-74.01167797',
 '-73.97498696',
 '-73.98789473',
 '-73.9919254',
 '-73.98169333',
 '-73.99138152',
 '-73.99422366',
 '-73.96023854',
 '-73.99074142',
 '-73.99372222',
 '-73.99190759',
 '-74.0046073',
 '-74.00297088',
 '-73.97371465',
 '-74.000271',
 '-73.98042047',
 '-73.96968902',
 '-73.98808416',
 '-73.98320928',
 '-74.00026394',
 '-73.98142006',
 '-73.97282625',
 '-74.00154702',
 '-73.988639',
 '-73.99138152',
 '-73.98320928',
 '-73.98564945',
 '-73.98848395',
 '-73.98788205',
 '-73.97207836',
 '-73.99800419',
 '-73.97768752',
 '-73.9907647',
 '-73.99949601',
 '-73.99379025',
 '-73.98362464',
 '-73.98338988',
 '-73.996375',
 '-74.01658354',
 '-74.00149746',
 '-74.0091059',
 '-74.0091059',
 '-73.96968902',
 '-74.00552427',
 '-73.9916',
 '-74.015756',
 '-73.978474',
 '-73.99301222',
 '-73.97995466',
 '-73.98442659',
 '-73.989111',
 '-73.99093085',
 